In [1]:
import numpy as np
import theano
import theano.tensor as T

In [2]:
from blocks.bricks import MLP
from blocks.bricks import Rectifier, Logistic
from blocks.bricks.cost import SquaredError
from blocks.graph import ComputationGraph
from blocks.algorithms import GradientDescent, RMSProp
from blocks.initialization import Uniform, Constant
from blocks.extensions.monitoring import DataStreamMonitoring
from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing

from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten

In [3]:
n_inpt = 25
n_hidden = 250
n_output = 64*64*3
mlp = MLP(activations=[Rectifier(), Logistic()],
         dims=[n_inpt, n_hidden, n_output],
         biases_init=Constant(0))

In [4]:
mlp.push_initialization_config()
mlp.children[0].weights_init = Uniform(
    width=8.*np.sqrt(6./(n_inpt+n_hidden))/255.)
mlp.children[2].weights_init = Uniform(
    width=8.*np.sqrt(6./(n_inpt+n_hidden)))

mlp.initialize()

In [5]:
x = T.matrix('label_features', dtype='floatX')
y_hat = mlp.apply(x)
y_raw = T.matrix('image_features', dtype='uint8')
y = T.cast(y_raw, dtype='floatX') / 255.

In [6]:
cost = SquaredError().apply(y, y_hat)
normalized_cost = cost / (64. * 64. * 3.)
cg = ComputationGraph(cost)

In [9]:
from fuel.datasets.hdf5 import H5PYDataset
train_set = H5PYDataset('../../../factory-robot-data/factory_dataset.hdf5', 
                        which_sets=('train',), 
                        subset=slice(0,5000))

test_set = H5PYDataset('../../../factory-robot-data/factory_dataset.hdf5',
                       which_sets=('test',))

data_stream_train = Flatten(DataStream.default_stream(
    train_set,
    iteration_scheme=SequentialScheme(train_set.num_examples, 
                                      batch_size=32)))

data_stream_monitor_test = Flatten(DataStream.default_stream(
    test_set,
    iteration_scheme=SequentialScheme(test_set.num_examples, 
                                      batch_size=1024)))

data_stream_monitor_train = Flatten(DataStream.default_stream(
    train_set,
    iteration_scheme=SequentialScheme(train_set.num_examples, 
                                      batch_size=1024)))

In [10]:
algorithm = GradientDescent(cost=cost, parameters=cg.parameters,
                             step_rule=RMSProp(learning_rate=0.1, 
                                              decay_rate=0.99))

In [11]:
monitor = DataStreamMonitoring(
    variables=[cost], data_stream=data_stream_monitor_test, prefix="test")
monitor2 = DataStreamMonitoring(
    variables=[cost], data_stream=data_stream_monitor_train, prefix="train")


In [12]:
main_loop = MainLoop(data_stream=data_stream_train, 
                     algorithm=algorithm,
                     extensions=[monitor, 
                                 monitor2,
                                 FinishAfter(after_n_epochs=5), 
                                 Printing()])

In [13]:
main_loop.run() 


Blocks will complete this epoch iteration of training and run extensions before exiting. If you do not want to complete this epoch, press CTRL + C again to stop training after the current batch.

Blocks will complete the current batch and run extensions before exiting. If you do not want to complete this batch, press CTRL + C again. WARNING: Note that this will end training immediately, and extensions that e.g. save your training progress won't be run.



-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: True
	 epoch_interrupt_received: True
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:
	 batch_interrupt_received: True
	 epoch_interrupt_received: True
	 test_squarederror_apply_cost: 546.197271139
	 train_squarederror_apply_cost: 548.65998736


-------------------------------------------------------------------------------
TRAINING HAS BEEN FINISHED:
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: True
	 epoch_interrupt_received: True
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 1
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
L

In [ ]:
epoch = data_stream.get_epoch_iterator()

In [ ]:
ex_image, ex_label = next(epoch)

cost.eval({x: ex_label, y_raw: ex_image})